In [4]:
import sys, os
pardir = os.path.abspath(os.path.join(os.path.dirname('__file__'), os.path.pardir))
sys.path.append(pardir)

import numpy as np
import librosa
import librosa.display
from scipy.io import wavfile
from aubio import onset
from scipy.signal import butter, lfilter
import matplotlib.pyplot as plt
import dsp

from read_labels import read_labels

%matplotlib inline

In [5]:
rootdir = '/home/tracek/Data/Birdman/'
filename = os.path.join(rootdir, 'raw/STHELENA-02_20140605_200000_1.wav')
outdir = os.path.join(rootdir, 'raw/samples/')
sheets = read_labels('/home/tracek/Data/Birdman/labels/sthelena_labels.xls')
filename_noext = os.path.splitext(os.path.basename(filename))[0]

# in seconds [s]
signal_start_s = 0
signal_end_s = 60

sheet = sheets[filename_noext]
condition = (sheet['Time Start'] > signal_start_s) & (sheet['Time End'] < signal_end_s)
sheet_selected = sheet[condition]

nfft = 512 # samples
hop = nfft // 2

/home/tracek/.virtualenvs/ml/lib/python3.5/site-packages/pandas/util/_decorators.py:118: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [7]:
y, sr = librosa.load(filename, sr=None)
y = y[signal_start_s * sr: signal_end_s * sr]
yh = dsp.highpass_filter(y, sr, highcut=500)

In [8]:
def get_onsets(signal, sr, nfft, hop, unit='s'):
    onsets = []

    onset_detector_type = 'hfc'
    onset_threshold = 0.01
    onset_silence_threshold = -70

    min_duration_s = 0.300

    onset_detector = onset(onset_detector_type, nfft, hop, sr)
    onset_detector.set_threshold(onset_threshold)
    onset_detector.set_silence(onset_silence_threshold)
    onset_detector.set_minioi_s(min_duration_s)

    signal_windowed = np.array_split(signal, np.arange(hop, len(signal), hop))

    for frame in signal_windowed[:-1]:
        if onset_detector(frame):
            if unit in {'seconds', 'second', 's'}:
                onsets.append(onset_detector.get_last_s())
            elif unit in {'sample', 'samples'}:
                onsets.append(onset_detector.get_last())
            else:
                raise ValueError('Unknown unit provided to the onset detector')
                
    return onsets

In [9]:
onsets_forward = np.array(get_onsets(yh, sr, nfft, hop, unit='s')[1:])
onsets_backward = np.array(get_onsets(yh[::-1], sr, nfft, hop)[1:])
onsets_backward_reversed = (signal_end_s - onsets_backward)[::-1]

In [10]:
slices = []

for onset_fw in onsets_forward:
    nearest_onset_backward_condition = onset_fw < onsets_backward_reversed - 0.2
    if nearest_onset_backward_condition[-1] == False:
        break
    else:
        idx_onset_backward = np.argmax(nearest_onset_backward_condition)
        slices.append((onset_fw, onsets_backward_reversed[idx_onset_backward]))

In [11]:
slices

[(2.9632499217987061, 3.3673133850097656),
 (3.3401250839233398, 3.7081260681152344),
 (3.7916250228881836, 4.1085014343261719),
 (8.123687744140625, 8.7581253051757812),
 (8.4540004730224609, 8.7581253051757812),
 (8.8272495269775391, 9.113311767578125),
 (9.9928121566772461, 10.388938903808594),
 (10.357687950134277, 10.853939056396484),
 (10.763562202453613, 11.250438690185547),
 (11.162625312805176, 14.826187133789062),
 (14.634312629699707, 15.208812713623047),
 (15.103750228881836, 15.58856201171875),
 (15.483875274658203, 16.984935760498047),
 (16.956249237060547, 17.469001770019531),
 (17.320999145507812, 17.859188079833984),
 (17.647811889648438, 17.859188079833984),
 (17.968936920166016, 18.246498107910156),
 (20.328937530517578, 20.920936584472656),
 (20.638063430786133, 20.920936584472656),
 (23.156000137329102, 23.477935791015625),
 (30.650312423706055, 31.509937286376953),
 (31.048500061035156, 31.509937286376953),
 (32.215126037597656, 32.770437240600586),
 (33.756126403